In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Asthma/GSE184382'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Sputum microRNA‐screening reveals Prostaglandin EP3 receptor as selective target in allergen‐specific immunotherapy"
!Series_summary	"Several microRNAs (miRs) have been described as potential biomarkers in liquid biopsies and in the context of allergic asthma, while therapeutic effects on the airway expression of miRs remain elusive. In this study, we investigated epigenetic miR-associated mechanisms in the sputum of grass pollen allergic patients with and without allergen specific immunotherapy (AIT). Induced sputum samples of healthy controls (HC), AIT treated and untreated grass pollen allergic rhinitis patients with (AA) and without asthma (AR) were profiled using miR microarray and transcriptome microarray analysis of the same samples. miR targets were predicted in silico and used to identify inverse regulation. Local PGE2 levels were measured using ELISA."
!Series_summary	"Two Hundred and fifty nine miRs were upregulated in the sputum of AA 

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# Check for gene expression data availability
is_gene_available = True  # Since there is transcriptome microarray analysis mentioned.

# Define trait row key after checking the data
trait_row = 1  # Asthma condition inferred from AIT treatment: yes/no

# Age and gender data not available
# As per the provided sample characteristics dictionary, there is no clear key for age or gender data.
age_row = None
gender_row = None

# Define functions for converting data
def convert_trait(value):
    if 'yes' in value.lower():
        return 1
    elif 'no' in value.lower():
        return 0
    else:
        return None

# Since age and gender data are not available, their converters are not needed
def convert_age(value):
    return None  # placeholder

def convert_gender(value):
    return None  # placeholder

# Save metadata
save_cohort_info('GSE184382', './preprocessed/Asthma/cohort_info.json', is_gene_available, trait_row is not None)

# Clinical Feature Extraction
if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Asthma', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Asthma/trait_data/GSE184382.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))


{'GSM5585358': [1], 'GSM5585359': [0], 'GSM5585360': [0], 'GSM5585361': [0], 'GSM5585362': [0], 'GSM5585363': [0], 'GSM5585364': [0], 'GSM5585365': [0], 'GSM5585366': [0], 'GSM5585367': [1], 'GSM5585368': [1], 'GSM5585369': [0], 'GSM5585370': [1], 'GSM5585371': [1], 'GSM5585372': [0], 'GSM5585373': [0], 'GSM5585374': [1], 'GSM5585375': [1], 'GSM5585376': [0], 'GSM5585377': [0], 'GSM5585378': [1], 'GSM5585379': [1], 'GSM5585380': [0], 'GSM5585381': [1], 'GSM5585382': [1], 'GSM5585383': [0], 'GSM5585384': [0], 'GSM5585385': [0], 'GSM5585386': [1], 'GSM5585387': [0], 'GSM5585388': [1], 'GSM5585389': [0], 'GSM5585390': [0], 'GSM5585391': [1], 'GSM5585392': [1], 'GSM5585393': [0], 'GSM5585394': [0], 'GSM5585395': [0], 'GSM5585396': [1]}


### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row ids for the following step.
print(gene_data.index[:20])


Index(['(+)E1A_r60_1', '(+)E1A_r60_3', '(+)E1A_r60_a104', '(+)E1A_r60_a107',
       '(+)E1A_r60_a135', '(+)E1A_r60_a20', '(+)E1A_r60_a22', '(+)E1A_r60_a97',
       '(+)E1A_r60_n11', '(+)E1A_r60_n9', '3xSLv1', 'A_19_P00315452',
       'A_19_P00315492', 'A_19_P00315493', 'A_19_P00315502', 'A_19_P00315506',
       'A_19_P00315518', 'A_19_P00315519', 'A_19_P00315529', 'A_19_P00315541'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
requires_gene_mapping = True


### Step 5: Gene Annotation (Conditional)

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the soft file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['GE_BrightCorner', 'DarkCorner', 'A_21_P0014386', 'A_33_P3396872', 'A_33_P3267760'], 'CONTROL_TYPE': ['pos', 'pos', 'FALSE', 'FALSE', 'FALSE'], 'REFSEQ': [nan, nan, nan, 'NM_001105533', nan], 'GB_ACC': [nan, nan, nan, 'NM_001105533', nan], 'LOCUSLINK_ID': [nan, nan, nan, 79974.0, 54880.0], 'GENE_SYMBOL': [nan, nan, nan, 'CPED1', 'BCOR'], 'GENE_NAME': [nan, nan, nan, 'cadherin-like and PC-esterase domain containing 1', 'BCL6 corepressor'], 'UNIGENE_ID': [nan, nan, nan, 'Hs.189652', nan], 'ENSEMBL_ID': [nan, nan, nan, nan, 'ENST00000378463'], 'ACCESSION_STRING': [nan, nan, nan, 'ref|NM_001105533|gb|AK025639|gb|BC030538|tc|THC2601673', 'ens|ENST00000378463'], 'CHROMOSOMAL_LOCATION': [nan, nan, 'unmapped', 'chr7:120901888-120901947', 'chrX:39909128-39909069'], 'CYTOBAND': [nan, nan, nan, 'hs|7q31.31', 'hs|Xp11.4'], 'DESCRIPTION': [nan, nan, nan, 'Homo sapiens cadherin-like and PC-esterase domain containing 1 (CPED1), transcript variant 2, mRNA [NM_001105533

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identifiers and gene symbols keys.
identifier_key = 'ID'
gene_symbol_key = 'GENE_SYMBOL'

# 2. Get the dataframe storing the mapping between probe IDs and genes
gene_mapping = get_gene_mapping(gene_annotation, prob_col=identifier_key, gene_col=gene_symbol_key)

# 3. Apply the mapping, resulting in the gene expression dataframe "gene_data"
gene_data = apply_gene_mapping(gene_data, gene_mapping)

# Handle any potential missing values explicitly
gene_data.dropna(inplace=True)

# Print a sample to verify the resulting gene_data (optional)
print(preview_df(gene_data))


{'GSM5585358': [-1.296134, -2.018902, -0.25098753, -2.1092324, -0.484729885], 'GSM5585359': [0.643692, 0.52343035, -1.2841356, 1.2390933, 0.42484975], 'GSM5585360': [-0.061505795, -0.36383057, 0.0, -1.6818285, -0.06618607500000001], 'GSM5585361': [-0.20103979, -0.2287507, -1.2330654, 0.28260612, -0.828105], 'GSM5585362': [0.63141966, -0.11431217, -1.5136883, -2.5823727, 0.04632211], 'GSM5585363': [0.32237053, -0.061508656, -1.2035153, 0.73365736, 0.13465594999999997], 'GSM5585364': [0.28095627, 0.0, -0.85097957, -0.15957594, 0.51199651], 'GSM5585365': [-0.02743721, 0.15571833, -0.51789117, 0.39014387, 0.19463611], 'GSM5585366': [-0.43631792, -0.6677952, 0.11063981, -1.4650202, -0.64131737], 'GSM5585367': [-0.033367634, 0.1772604, 0.31470847, 0.28247452, 0.11428951500000001], 'GSM5585368': [0.3456807, 0.5995779, 1.2547424, 0.043848038, 0.326348545], 'GSM5585369': [-0.3327813, -0.9190221, -2.3481348, -0.7244148, -0.84109378], 'GSM5585370': [0.032586575, -0.28737736, 0.5591204, -0.5971441

### Step 7: Data Normalization and Merging

In [8]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
gene_csv_path = './preprocessed/Asthma/gene_data/GSE184382.csv'
normalized_gene_data.to_csv(gene_csv_path)

# 2. Merge the clinical and genetic data with the 'geo_merge_clinical_genetic_data' function from the library.
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, normalized_gene_data)

# 3. Determine whether the trait and some demographic attributes in the data is severely biased, and remove biased attributes.
trait_biased, unbiased_merged_data = judge_and_remove_biased_features(merged_data, 'Asthma')

# If the trait is not severely biased, save the cohort information and the merged data.

# 4. Save the cohort information.
save_cohort_info('GSE184382', './preprocessed/Asthma/cohort_info.json', True, True, trait_biased, merged_data)

if not trait_biased:
    # 5. If the trait is not severely biased, save the merged data to a csv file.
    csv_path = './preprocessed/Asthma/GSE184382.csv'
    unbiased_merged_data.to_csv(csv_path)


For the feature 'Asthma', the least common label is '1.0' with 16 occurrences. This represents 41.03% of the dataset.
The distribution of the feature 'Asthma' in this dataset is fine.

